<a href="https://colab.research.google.com/github/tzf101/190041101-CSE-4302/blob/master/utils_notebook/bt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libraries

### Libraries

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install git+https://github.com/csebuetnlp/normalizer

In [ ]:
!pip install bnaug

In [4]:
!pip install -U sentence-transformers
!pip install sacrebleu
!pip install bert-score
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4b40571c777c1124a022f9bbe8760444e29f51a880fae5e6c49ae1784c0c3945
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00


In [5]:
import random
import re
import pandas as pd
import sacrebleu
from sacrebleu import corpus_bleu
from bert_score import score
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu


In [6]:
!pip install bnlp-toolkit

In [12]:
from bnlp import corpus
from bnlp import BengaliWord2Vec

### Load Model Bangla to English

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize

modelb2e = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_bn_en")
tokenizerb2e = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_bn_en", use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [14]:
def bn_to_en(input_sentence):

    input_ids = tokenizerb2e(normalize(input_sentence), return_tensors="pt").input_ids
    generated_tokens = modelb2e.generate(input_ids)
    decoded_tokens = tokenizerb2e.batch_decode(generated_tokens)[0]

    return decoded_tokens

In [15]:
text = "সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।"
translated_sentence = bn_to_en(text)
print(translated_sentence)

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> A proper investigation should be done, and the person who did this should be brought to justice.</s>


### Load Model English to Bangla

In [16]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_en_bn")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_en_bn", use_fast=False)

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [17]:
def en_to_bn(input_sentence):
    input_ids = tokenizer(normalize(input_sentence), return_tensors="pt").input_ids
    generated_tokens = model.generate(input_ids)
    decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]

    return decoded_tokens

In [18]:
output_sentence = en_to_bn(translated_sentence)
print(output_sentence)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:219: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


<pad> সঠিক তদন্ত হওয়া উচিত এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনা উচিত।</s>


### Loading SBERT Model

In [19]:
sbert_model = SentenceTransformer('l3cube-pune/bengali-sentence-bert-nli')

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

# BT Code

### BT Class

In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import random
from bnlp import BasicTokenizer
from normalizer import normalize

class BaseTextAugmentor:
    def __init__(self, clean_text):
        self.clean_text = clean_text

    def augment(self, text):
        raise NotImplementedError("This method should be implemented by subclasses")
class BTAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, modelb2e_name="csebuetnlp/banglat5_nmt_bn_en", modele2b_name="csebuetnlp/banglat5_nmt_en_bn", top_k=1, top_p=1, num_beams=1):
        super().__init__(clean_text)
        self.modelb2e = AutoModelForSeq2SeqLM.from_pretrained(modelb2e_name)
        self.tokenizerb2e = AutoTokenizer.from_pretrained(modelb2e_name, use_fast=False)

        self.modele2b = AutoModelForSeq2SeqLM.from_pretrained(modele2b_name)
        self.tokenizere2b = AutoTokenizer.from_pretrained(modele2b_name, use_fast=False)

        self.top_k = top_k
        self.top_p = top_p
        self.num_beams = num_beams

    def augment(self, text, debug=False):
        # Encode the Bangla text
        input_ids = self.tokenizerb2e.encode(normalize(text), return_tensors="pt")

        # Generate English translation
        en_outputs = self.modelb2e.generate(
            input_ids,
            # top_k=self.top_k,
            # top_p=self.top_p,
            # num_beams=self.num_beams
        )

        # Decode the English text
        en_text = self.tokenizerb2e.decode(en_outputs[0], skip_special_tokens=True)

        # Encode the English text
        input_ids = self.tokenizere2b.encode(normalize(en_text), return_tensors="pt")

        # Generate Bangla translation
        bn_outputs = self.modele2b.generate(
            input_ids,
            # top_k=self.top_k,
            # top_p=self.top_p,
            # num_beams=self.num_beams
        )

        # Decode the final Bangla text
        bn_text = self.tokenizere2b.decode(bn_outputs[0], skip_special_tokens=True)

        if debug:
            bn_text += '(bt)'

        return bn_text

# Example usage
bt_augmentor = BTAugmentor(clean_text="some text")
augmented_text = bt_augmentor.augment("সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে")
print(augmented_text)


সঠিক তদন্ত করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।


### Score Generator

In [46]:
def calculate_sbert_score(original, augmented):
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()
    return sbert_score

In [28]:

def calculate_scores(original, augmented):
    # BLEU Score
    reference = original
    candidate = augmented
    # bleu_score = sentence_bleu(reference, candidate)
    bleu_score = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(augmented, original)]
    # BERTScore
    P, R, F1 = score([augmented], [original], lang="bn", rescale_with_baseline=True)

    # SBERT Score with Cosine Similarity
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()

    return bleu_score, F1.item(), sbert_score

### Testing BT

In [22]:
import pandas as pd
from rouge import Rouge
import sacrebleu

# Sample data
data = {
    "sentence1": ["সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে", "তামিম থেকে সাকিব ভাল ক্রিকেট খেলে"],
    "label": [1, 0]
}
df = pd.DataFrame(data)

# Example text augmentation function (replace with your actual function)
def augment_text(text):
    # Your augmentation logic goes here. For now, returning the text as-is.
    return bt_augmentor.augment(text)

# Apply the augmentation function to the "sentence" column
df["augmented_sentence"] = df["sentence1"].apply(augment_text)

# Compute ROUGE scores
rouge = Rouge()
rouge_scores = rouge.get_scores(df["augmented_sentence"], df["sentence1"], avg=True)

# Compute SacreBLEU scores
sacrebleu_scores = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(df["augmented_sentence"], df["sentence1"])]


# Output
print("ROUGE Scores:", rouge_scores)
print("SacreBLEU Scores:", sacrebleu_scores)
print("\nAugmented DataFrame:\n", df)


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ROUGE Scores: {'rouge-1': {'r': 0.8012820512820513, 'p': 0.7738095238095238, 'f': 0.7870370320404665}, 'rouge-2': {'r': 0.3666666666666667, 'p': 0.35384615384615387, 'f': 0.3599999950040001}, 'rouge-l': {'r': 0.5641025641025641, 'p': 0.5476190476190476, 'f': 0.555555550558985}}
SacreBLEU Scores: [20.200106912694157, 24.274588585366175]

Augmented DataFrame:
                                            sentence1  label  \
0  সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...      1   
1                  তামিম থেকে সাকিব ভাল ক্রিকেট খেলে      0   

                                  augmented_sentence  
0  সঠিক তদন্ত করতে হবে এবং যে ব্যক্তি এটা করেছে ত...  
1                 সাকিব তামিম থেকে ভালো ক্রিকেট খেলে  


In [23]:
# Sample Bangla Sentiment Analysis Dataset
bangla_dataset = [
    {'sentence1': 'আমি আজ খুব খুশি।', 'label': 'positive'},
    {'sentence1': 'বৃষ্টির জন্য আমি ভালো মনে হয়নি।', 'label': 'negative'},
    {'sentence1': 'আজকের দিনটি সাধারিত।', 'label': 'neutral'},
    {'sentence1': 'এই বইটি আমি ভালোভাবে বুঝতে পারতেছি, কিন্তু কিছুটা জটিল।', 'label': 'mixed'},
    {'sentence1': 'পুজোয় বন্ধুবান্ধবী সাথে সময় কাটানো অসাধারণ ছিল।', 'label': 'positive'}
]

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(bangla_dataset)

# Display the DataFrame
df

,sentence1,label
0,আমি আজ খুব খুশি।,positive
1,বৃষ্টির জন্য আমি ভালো মনে হয়নি।,negative
2,আজকের দিনটি সাধারিত।,neutral
3,"এই বইটি আমি ভালোভাবে বুঝতে পারতেছি, কিন্তু কিছ...",mixed
4,পুজোয় বন্ধুবান্ধবী সাথে সময় কাটানো অসাধারণ ছিল।,positive


In [29]:
new_data = []
for index, row in df.iterrows():
    original_text = row['sentence1']
    label = row['label']
    augmented_text = bt_augmentor.augment(original_text)
    bleu, bert, sbert = calculate_scores(original_text, augmented_text)
    new_data.append({
        'sentence1': augmented_text,
        'original_text': original_text,
        'label': label,
        'SBLEU': bleu,
        'SBERT': sbert,
        'BERTScore': bert,
        'method': 'bt'
    })

new_df = pd.DataFrame(new_data)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.w

In [30]:
new_df

,sentence1,original_text,label,SBLEU,SBERT,BERTScore,method
0,আমি আজ খুব খুশি।,আমি আজ খুব খুশি।,positive,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.000000,1.000000,bt
1,বৃষ্টির জন্য আমার ভালো লাগছে না।,বৃষ্টির জন্য আমি ভালো মনে হয়নি।,negative,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.960330,0.879838,bt
2,আজ সেই দিন।,আজকের দিনটি সাধারিত।,neutral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.794379,0.736678,bt
3,"আমি এই বইটি ভালভাবে বুঝতে পারি, কিন্তু কিছুটা ...","এই বইটি আমি ভালোভাবে বুঝতে পারতেছি, কিন্তু কিছ...",mixed,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.960318,0.947566,bt
4,দুর্গাপূজায় বন্ধুদের সঙ্গে সময় কাটানো ছিল অস...,পুজোয় বন্ধুবান্ধবী সাথে সময় কাটানো অসাধারণ ছিল।,positive,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.887222,0.873654,bt


In [51]:
import pandas as pd
from rouge import Rouge
import sacrebleu

# Sample data
bangla_dataset = [
    {'sentence1': 'আমি আজ খুব খুশি।', 'label': 'positive'},
    {'sentence1': 'বৃষ্টির জন্য আমি ভালো মনে হয়নি।', 'label': 'negative'},
    {'sentence1': 'আজকের দিনটি সাধারিত।', 'label': 'neutral'},
    {'sentence1': 'এই বইটি আমি ভালোভাবে বুঝতে পারতেছি, কিন্তু কিছুটা জটিল।', 'label': 'mixed'},
    {'sentence1': 'পুজোয় বন্ধুবান্ধবী সাথে সময় কাটানো অসাধারণ ছিল।', 'label': 'positive'}
]

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(bangla_dataset)
df

,sentence1,label
0,আমি আজ খুব খুশি।,positive
1,বৃষ্টির জন্য আমি ভালো মনে হয়নি।,negative
2,আজকের দিনটি সাধারিত।,neutral
3,"এই বইটি আমি ভালোভাবে বুঝতে পারতেছি, কিন্তু কিছ...",mixed
4,পুজোয় বন্ধুবান্ধবী সাথে সময় কাটানো অসাধারণ ছিল।,positive


In [74]:
# Example text augmentation function (replace with your actual function)
def augment_text(text):
    # Your augmentation logic goes here. For now, returning the text as-is.
    return bt_augmentor.augment(text)

def calculate_scores(input_df):
    # Create a copy of the input DataFrame to avoid modifying the original DataFrame
    result_df = input_df.copy()
    result_df = result_df.rename(columns={'sentence1': 'original_sentence'})

    # Apply the augmentation function to the "sentence1" column
    result_df["augmented_sentence"] = result_df["original_sentence"].apply(augment_text)

    # Compute ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(result_df["augmented_sentence"], result_df["original_sentence"])
    rouge_df = pd.DataFrame([
    {
        'r1p': score['rouge-1']['p'],
        'r1r': score['rouge-1']['r'],
        'r1f': score['rouge-1']['f'],
        'r2p': score['rouge-2']['p'],
        'r2r': score['rouge-2']['r'],
        'r2f': score['rouge-2']['f'],
        'rlp': score['rouge-l']['p'],
        'rlr': score['rouge-l']['r'],
        'rlf': score['rouge-l']['f'],
    }
    for score in rouge_scores
])
    # Compute SacreBLEU scores
    sacrebleu_scores = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(result_df["augmented_sentence"], result_df["original_sentence"])]

    result_df["sacrebleu_score"] = sacrebleu_scores

    # SBERT Score with Cosine Similarity
    result_df["sbert_score"] = [calculate_sbert_score(orig, aug) for orig, aug in zip(result_df["original_sentence"], result_df["augmented_sentence"])]
    result_df = pd.concat([result_df, rouge_df], axis=1)
    result_df["method"] = "bt"

    return result_df

# Call the function to calculate scores and get the augmented DataFrame
result_df = calculate_scores(df)


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [75]:
# new_data = []
# for index, row in df.iterrows():
#     original_text = row['sentence1']
#     label = row['label']
#     augmented_text = bt_augmentor.augment(original_text)
#     bleu, bert, sbert = calculate_scores(original_text, augmented_text)
#     new_data.append({
#         'sentence1': augmented_text,
#         'original_text': original_text,
#         'label': label,
#         'SBLEU': bleu,
#         'SBERT': sbert,
#         'BERTScore': bert,
#         'method': 'bt'
#     })

# new_df = pd.DataFrame(new_data)

In [76]:
result_df

,original_sentence,label,augmented_sentence,sacrebleu_score,sbert_score,r1p,r1r,r1f,r2p,r2r,r2f,rlp,rlr,rlf,method
0,আমি আজ খুব খুশি।,positive,আমি আজ খুব খুশি।,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,bt
1,বৃষ্টির জন্য আমি ভালো মনে হয়নি।,negative,বৃষ্টির জন্য আমার ভালো লাগছে না।,17.965206,0.960330,0.500000,0.500000,0.500000,0.200000,0.200000,0.200000,0.500000,0.500000,0.500000,bt
2,আজকের দিনটি সাধারিত।,neutral,আজ সেই দিন।,0.000000,0.794379,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,bt
3,"এই বইটি আমি ভালোভাবে বুঝতে পারতেছি, কিন্তু কিছ...",mixed,"আমি এই বইটি ভালভাবে বুঝতে পারি, কিন্তু কিছুটা ...",33.568919,0.960318,0.777778,0.777778,0.777778,0.375000,0.375000,0.375000,0.666667,0.666667,0.666667,bt
4,পুজোয় বন্ধুবান্ধবী সাথে সময় কাটানো অসাধারণ ছিল।,positive,দুর্গাপূজায় বন্ধুদের সঙ্গে সময় কাটানো ছিল অস...,13.134549,0.887222,0.285714,0.285714,0.285714,0.166667,0.166667,0.166667,0.285714,0.285714,0.285714,bt
